<a href="https://colab.research.google.com/github/BruceGabr/Neurokup-III/blob/main/notebooks/03_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf ~/.kaggle           # Eliminar archivos antiguos
!rm -rf ~/.config/kaggle    # Eliminar configuraciones alternativas

import os
import json
for var in ['KAGGLE_USERNAME', 'KAGGLE_KEY', 'KAGGLE_API_TOKEN']:
    os.environ.pop(var, None)  # Eliminar variables de entorno viejas

In [2]:
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
# ========== 1. DEFINIR DIRECTORIO ==========
kaggle_dir = os.path.expanduser('~/.kaggle')
# ========== 3. CONFIGURACIÓN ==========
config = {
    "username": "brucemg",
    "key": "a09e6281740a6fe675766513592210d7"  # Reemplaza con tu key real
}

# Crear archivo NUEVO desde cero
with open(f'{kaggle_dir}/kaggle.json', 'w') as f:
    json.dump(config, f)

# Establecer variables de entorno NUEVAS
os.environ['KAGGLE_USERNAME'] = config['username']
os.environ['KAGGLE_KEY'] = config['key']

In [4]:
!kaggle config view

Configuration values from /root/.kaggle
- username: brucemg
- auth_method: LEGACY_API_KEY
- path: None
- proxy: None
- competition: None


In [5]:
!pip install -q kaggle

In [6]:
!kaggle competitions list | head

ref                                                                              deadline             category          reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-3    2026-04-15 23:59:00  Featured   2,207,152 Usd       1052           False  
https://www.kaggle.com/competitions/passenger-screening-algorithm-challenge      2017-12-15 23:59:00  Featured   1,500,000 Usd        518           False  
https://www.kaggle.com/competitions/zillow-prize-1                               2018-01-10 15:59:00  Featured   1,200,000 Usd       3770           False  
https://www.kaggle.com/competitions/data-science-bowl-2017                       2017-04-12 23:59:00  Featured   1,000,000 Usd       1972           False  
https://www.kaggle.com/competitions/vesuvius-challenge-ink-detec

# Cargar datos

In [7]:
!kaggle competitions download -c neuro-kup-iii-beta
!unzip neuro-kup-iii-beta.zip

neuro-kup-iii-beta.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  neuro-kup-iii-beta.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Importar librerías

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from IPython.display import display

# Preprocesamiento

### Leer datasets

In [9]:
train = pd.read_csv('train.csv')
test_private = pd.read_csv('test_private.csv')
test_public = pd.read_csv('test_public.csv')

### Crear copias d elos datasets

In [10]:
train_copy = train.copy()
test_private_copy = test_private.copy()
test_public_copy = test_public.copy()

### Eliminar columnas que no aportan

In [11]:
import re

def duracion_a_segundos(x):
    if pd.isna(x):
        return 0

    x = str(x)
    minutos = re.search(r"(\d+)\s*m", x)
    segundos = re.search(r"(\d+)\s*s", x)

    m = int(minutos.group(1)) if minutos else 0
    s = int(segundos.group(1)) if segundos else 0

    return m * 60 + s

def limpiar_p_dias(x):
    if str(x).lower() == "nunca":
        return -1
    return int(x)

In [12]:
def pre_process(df):
  drop_cols = ["id", "nombre", "apellido", "DNI"]
  df_copy = df.drop(columns=drop_cols)
  df_copy["duracion"] = df_copy["duracion"].apply(duracion_a_segundos)
  df_copy["p_dias"] = df_copy["p_dias"].apply(limpiar_p_dias)
  if "y" in df_copy.columns:
    df_copy['y'] = df_copy['y'].map({'si': 1, 'no': 0})
  return df_copy


In [13]:
train_copy = pre_process(train_copy)
test_private_copy = pre_process(test_private_copy)
test_public_copy = pre_process(test_public_copy)

In [14]:
train_copy.info()
test_public_copy.info()
test_private_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36168 entries, 0 to 36167
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   año_nacimiento         36168 non-null  int64  
 1   trabajo                36168 non-null  object 
 2   estado_civil           36168 non-null  object 
 3   educación              36168 non-null  object 
 4   riesgo_crediticio      36168 non-null  object 
 5   fondos_promedio_anual  36168 non-null  float64
 6   hipoteca               36168 non-null  object 
 7   deuda_personal         36168 non-null  object 
 8   incumplimiento         36168 non-null  object 
 9   contacto               36168 non-null  object 
 10  fecha                  36168 non-null  object 
 11  duracion               36168 non-null  int64  
 12  campaña                36168 non-null  int64  
 13  p_dias                 36168 non-null  int64  
 14  contactos_previos      36168 non-null  int64  
 15  p_

In [15]:
train_copy["y"].value_counts()

,count
y,
0,31970
1,4198


### Preprocesar columna "trabajo"

In [16]:
"""
ESTANDARIZACIÓN - COLUMNA 'trabajo'
Solo unificar mayúsculas/minúsculas y variaciones ortográficas
"""

import pandas as pd

# ========== FUNCIÓN DE ESTANDARIZACIÓN ==========
def estandarizar_trabajo(df):
    """
    Unifica solo variaciones de escritura de la misma categoría
    """
    df = df.copy()

    # 1. Convertir todo a minúsculas y quitar espacios extras
    df['trabajo'] = df['trabajo'].str.lower().str.strip()

    # 2. Mapeo SOLO de variaciones ortográficas
    mapeo_estandarizacion = {
        # Técnico (con y sin tilde)
        'tecnico': 'técnico',

        # Autónomo (con y sin tilde, con typo)
        'autónomo': 'autónomo',
        'autonomo': 'autónomo',  # sin tilde

        # Criado/a (unificar formato)
        'criada(o)': 'criado/a',
        'criado(a)': 'criado/a',

        # Servicio/servicios (unificar plural)
        'servicios': 'servicio',

        # Desconocidos (unificar todas las formas)
        'no se conoce': 'desconocido',
        'sin información': 'desconocido',
        'no se sabe': 'desconocido',
    }

    # 3. Aplicar mapeo
    df['trabajo'] = df['trabajo'].replace(mapeo_estandarizacion)

    return df

# ========== APLICAR ==========
train_clean = estandarizar_trabajo(train_copy)
test_public_clean = estandarizar_trabajo(test_public_copy)
test_private_clean = estandarizar_trabajo(test_private_copy)

print("Distribución DESPUÉS (estandarizada):")
print(train_clean['trabajo'].value_counts())
print(f"\n   Categorías finales: {train_clean['trabajo'].nunique()}")

print(f"\nReducción: {train_clean['trabajo'].nunique()} categorías")

Distribución DESPUÉS (estandarizada):
trabajo
obrero             7806
gestión            7500
técnico            5131
servicio           3350
administrativo     2904
autónomo           1286
administrador      1233
emprendedor        1178
criado/a           1002
experto técnico     965
jubilado            915
retirado            900
desempleado         805
estudiante          742
desconocido         233
desocupado          218
Name: count, dtype: int64

   Categorías finales: 16

Reducción: 16 categorías


In [17]:
"""
PASO 2: UNIFICAR SINÓNIMOS
"""

def unificar_sinonimos_trabajo(df):
    """
    Unifica categorías que son exactamente lo mismo
    """
    df = df.copy()

    # Mapeo de sinónimos
    sinonimos = {
        'retirado': 'jubilado',          # Mismo significado
        'desocupado': 'desempleado',     # Mismo significado
    }

    df['trabajo'] = df['trabajo'].replace(sinonimos)

    return df

# Aplicar
train_clean = unificar_sinonimos_trabajo(train_clean)
test_public_clean = unificar_sinonimos_trabajo(test_public_clean)
test_private_clean = unificar_sinonimos_trabajo(test_private_clean)

# Verificar
print("Sinónimos unificados\n")
print("Distribución final:")
print(train_clean['trabajo'].value_counts())
print(f"\nCategorías finales: {train_clean['trabajo'].nunique()}")

Sinónimos unificados

Distribución final:
trabajo
obrero             7806
gestión            7500
técnico            5131
servicio           3350
administrativo     2904
jubilado           1815
autónomo           1286
administrador      1233
emprendedor        1178
desempleado        1023
criado/a           1002
experto técnico     965
estudiante          742
desconocido         233
Name: count, dtype: int64

Categorías finales: 14


In [18]:
display(train_clean.head())

,año_nacimiento,trabajo,estado_civil,educación,riesgo_crediticio,fondos_promedio_anual,hipoteca,deuda_personal,incumplimiento,contacto,fecha,duracion,campaña,p_dias,contactos_previos,p_resultado,y
0,1984,obrero,"casado, con hijos",primaria,no,849.0,sí,Falso,NO,Desconocido,15-may,72,1,-1,0,desconocido,0
1,1976,técnico,"casado, sin hijos",primaria,no,1415.0,sí,Falso,NO,CEL,30-jul,269,2,-1,0,desconocido,0
2,1983,administrador,"casado, con hijos",secundaria,no,3842.0,no,Falso,NO,CEL,31-jul,130,4,-1,0,Desconocido,0
3,1988,gestión,soltero,superios,no,-119.0,sí,Falso,NO,desconocido,11-ene,375,11,-1,0,desconocido,0
4,1969,obrero,"casado, con hijos",primaria,no,3498.0,No,Falso,NO,CEL,15-abr,264,2,-1,0,desconocido,1


### Preprocesar columna "estado_civil"

In [19]:
"""
=================================================================
PREPROCESAMIENTO - COLUMNA 'estado_civil'
=================================================================
"""

def limpiar_estado_civil(df):
    """
    Limpia la columna 'estado_civil'
    - Normaliza formato
    - Unifica tipos de divorcio
    """
    df = df.copy()

    # Normalizar (minúsculas, espacios)
    df['estado_civil'] = df['estado_civil'].str.lower().str.strip()

    # Unificar divorcios
    mapeo = {
        'divorcio voluntario': 'divorciado',
        'divorcio contencioso': 'divorciado',
    }

    df['estado_civil'] = df['estado_civil'].replace(mapeo)

    return df

# ========== APLICAR ==========
train_clean = limpiar_estado_civil(train_clean)
test_public_clean = limpiar_estado_civil(test_public_clean)
test_private_clean = limpiar_estado_civil(test_private_clean)

# ========== VERIFICAR ==========
print("✅ Columna 'estado_civil' limpia")
print(f"📊 Categorías: {train_clean['estado_civil'].nunique()}")
print("\n📋 Distribución:")
print(train_clean['estado_civil'].value_counts())

✅ Columna 'estado_civil' limpia
📊 Categorías: 4

📋 Distribución:
estado_civil
casado, sin hijos    10918
casado, con hijos    10886
soltero              10221
divorciado            4143
Name: count, dtype: int64


### Preprocesar columna "educación"

In [20]:
"""
=================================================================
PREPROCESAMIENTO - COLUMNA 'educación'
=================================================================
"""

def limpiar_educacion(df):
    """
    Limpia y estandariza la columna 'educación'

    Cambios:
    - Normaliza a minúsculas
    - Corrige typos: 'superios' → 'superior', 'primari@' → 'primaria'
    - Unifica valores desconocidos → 'desconocido'

    Returns:
        df: DataFrame con columna limpia (4 categorías)
    """
    df = df.copy()

    # Normalizar formato (minúsculas, espacios)
    df['educación'] = df['educación'].str.lower().str.strip()

    # Corregir typos y unificar
    mapeo = {
        # Typos ortográficos
        'superios': 'superior',
        'primari@': 'primaria',

        # Valores desconocidos
        'sin información': 'desconocido',
        'no se conoce': 'desconocido',
        'no se sabe': 'desconocido',
    }

    df['educación'] = df['educación'].replace(mapeo)

    return df

# ========== APLICAR ==========
train_clean = limpiar_educacion(train_clean)
test_public_clean = limpiar_educacion(test_public_clean)
test_private_clean = limpiar_educacion(test_private_clean)

# ========== VERIFICAR ==========
print("="*60)
print("LIMPIEZA COLUMNA 'educación' - COMPLETADA")
print("="*60)

print(f"\n📊 Categorías: 10 → {train_clean['educación'].nunique()}")
print("\n📋 Distribución final:")
print(train_clean['educación'].value_counts())

print("\n✅ Preprocesamiento completo")


LIMPIEZA COLUMNA 'educación' - COMPLETADA

📊 Categorías: 10 → 4

📋 Distribución final:
educación
secundaria     18670
superior       10539
primaria        5476
desconocido     1483
Name: count, dtype: int64

✅ Preprocesamiento completo


### Preprocesar columna "riesgo_crediticio"

In [21]:
# Convertir a booleano
train_clean['tiene_riesgo'] = train_clean['riesgo_crediticio'].map({'no': 0, 'sí': 1})
test_public_clean['tiene_riesgo'] = test_public_clean['riesgo_crediticio'].map({'no': 0, 'sí': 1})
test_private_clean['tiene_riesgo'] = test_private_clean['riesgo_crediticio'].map({'no': 0, 'sí': 1})

In [22]:
# Luego eliminar la original
train_clean = train_clean.drop('riesgo_crediticio', axis=1)
test_public_clean = test_public_clean.drop('riesgo_crediticio', axis=1)
test_private_clean = test_private_clean.drop('riesgo_crediticio', axis=1)

In [23]:
print(f"\n tiene_riesgo:")
print(train_clean["tiene_riesgo"].value_counts())
print(f"Valores únicos: {train_clean["tiene_riesgo"].nunique()}")


 tiene_riesgo:
tiene_riesgo
0    35491
1      677
Name: count, dtype: int64
Valores únicos: 2


### Preprocesar columna "hipoteca"

In [24]:
"""
=================================================================
PREPROCESAMIENTO - COLUMNA 'hipoteca'
=================================================================
"""

def limpiar_hipoteca(df):
    """
    Limpia y estandariza la columna 'hipoteca'

    Cambios:
    - Normaliza a minúsculas
    - Unifica 'Sí'/'sí' → 'sí'
    - Unifica 'No'/'no' → 'no'

    Returns:
        df: DataFrame con columna limpia (2 categorías)
    """
    df = df.copy()

    # Normalizar a minúsculas
    df['hipoteca'] = df['hipoteca'].str.lower().str.strip()

    return df

# ========== APLICAR ==========
train_clean = limpiar_hipoteca(train_clean)
test_public_clean = limpiar_hipoteca(test_public_clean)
test_private_clean = limpiar_hipoteca(test_private_clean)

# ========== VERIFICAR ==========
print("="*60)
print("LIMPIEZA COLUMNA 'hipoteca' - COMPLETADA")
print("="*60)

print(f"\n📊 Categorías: 4 → {train_clean['hipoteca'].nunique()}")
print("\n📋 Distribución final:")
print(train_clean['hipoteca'].value_counts())

# Verificar valores
valores_esperados = {'sí', 'no'}
valores_reales = set(train_clean['hipoteca'].unique())

if valores_reales == valores_esperados:
    print("\n✅ Limpieza exitosa - 2 categorías correctas")
else:
    print(f"\n⚠️  Valores inesperados: {valores_reales}")

# Verificar balance
print(f"\n📊 Distribución porcentual:")
print(train_clean['hipoteca'].value_counts(normalize=True) * 100)

LIMPIEZA COLUMNA 'hipoteca' - COMPLETADA

📊 Categorías: 4 → 2

📋 Distribución final:
hipoteca
sí    20135
no    16033
Name: count, dtype: int64

✅ Limpieza exitosa - 2 categorías correctas

📊 Distribución porcentual:
hipoteca
sí    55.670759
no    44.329241
Name: proportion, dtype: float64


In [25]:
# Convertir a binario (0/1)
train_clean['tiene_hipoteca'] = train_clean['hipoteca'].map({'no': 0, 'sí': 1})
test_public_clean['tiene_hipoteca'] = test_public_clean['hipoteca'].map({'no': 0, 'sí': 1})
test_private_clean['tiene_hipoteca'] = test_private_clean['hipoteca'].map({'no': 0, 'sí': 1})

train_clean = train_clean.drop('hipoteca', axis=1)
test_public_clean = test_public_clean.drop('hipoteca', axis=1)
test_private_clean = test_private_clean.drop('hipoteca', axis=1)

In [26]:
# Verificar balance
print(f"\n📊 Distribución porcentual:")
print(train_clean['tiene_hipoteca'].value_counts(normalize=True) * 100)


📊 Distribución porcentual:
tiene_hipoteca
1    55.670759
0    44.329241
Name: proportion, dtype: float64


### Preprocesar columna "deuda_personal"

In [27]:
# Opción A: Booleano (True/False)
train_clean['deuda_personal'] = train_clean['deuda_personal'].map({
    'Falso': False,
    'Verdadero': True
})
test_public_clean['deuda_personal'] = test_public_clean['deuda_personal'].map({
    'Falso': False,
    'Verdadero': True
})
test_private_clean['deuda_personal'] = test_private_clean['deuda_personal'].map({
    'Falso': False,
    'Verdadero': True
})
# Verificar balance
print(f"\n📊 Distribución porcentual:")
print(train_clean['deuda_personal'].value_counts(normalize=True) * 100)


📊 Distribución porcentual:
deuda_personal
False    83.866954
True     16.133046
Name: proportion, dtype: float64


### Preprocesar columna "incumplimiento"

In [28]:
"""
VERIFICACIÓN CORRECTA DE COLUMNAS DUPLICADAS
"""

# Convertir a numérico
train_clean['incumplimiento'] = train_clean['incumplimiento'].map({'NO': 0, 'SÍ': 1})
test_public_clean['incumplimiento'] = test_public_clean['incumplimiento'].map({'NO': 0, 'SÍ': 1})
test_private_clean['incumplimiento'] = test_private_clean['incumplimiento'].map({'NO': 0, 'SÍ': 1})

# ========== FORMA CORRECTA: Comparar valores fila por fila ==========

# Verificar en cada dataset
print("="*60)
print("VERIFICACIÓN DE DUPLICADOS")
print("="*60)

# Train
train_iguales = (train_clean['tiene_riesgo'] == train_clean['incumplimiento']).all()
print(f"\n📊 TRAIN:")
print(f"   ¿Son idénticas? {train_iguales}")
if not train_iguales:
    diferentes = (train_clean['tiene_riesgo'] != train_clean['incumplimiento']).sum()
    print(f"   Filas diferentes: {diferentes} ({diferentes/len(train_clean)*100:.2f}%)")

# Test Public
test_pub_iguales = (test_public_clean['tiene_riesgo'] == test_public_clean['incumplimiento']).all()
print(f"\n📊 TEST PUBLIC:")
print(f"   ¿Son idénticas? {test_pub_iguales}")
if not test_pub_iguales:
    diferentes = (test_public_clean['tiene_riesgo'] != test_public_clean['incumplimiento']).sum()
    print(f"   Filas diferentes: {diferentes} ({diferentes/len(test_public_clean)*100:.2f}%)")

# Test Private
test_priv_iguales = (test_private_clean['tiene_riesgo'] == test_private_clean['incumplimiento']).all()
print(f"\n📊 TEST PRIVATE:")
print(f"   ¿Son idénticas? {test_priv_iguales}")
if not test_priv_iguales:
    diferentes = (test_private_clean['tiene_riesgo'] != test_private_clean['incumplimiento']).sum()
    print(f"   Filas diferentes: {diferentes} ({diferentes/len(test_private_clean)*100:.2f}%)")

# ========== DECISIÓN ==========
print("\n" + "="*60)
print("DECISIÓN")
print("="*60)

if train_iguales and test_pub_iguales and test_priv_iguales:
    print("\n🚨 COLUMNAS DUPLICADAS en TODOS los datasets")
    print("   → Eliminando 'incumplimiento'...")

    # Eliminar en todos
    train_clean = train_clean.drop('incumplimiento', axis=1)
    test_public_clean = test_public_clean.drop('incumplimiento', axis=1)
    test_private_clean = test_private_clean.drop('incumplimiento', axis=1)

    print("   ✅ Columna 'incumplimiento' eliminada de todos los datasets")
    print("   ✅ Mantenida columna 'tiene_riesgo'")
else:
    print("\n✅ Las columnas son diferentes en al menos un dataset")
    print("   → Se mantienen ambas columnas")

VERIFICACIÓN DE DUPLICADOS

📊 TRAIN:
   ¿Son idénticas? True

📊 TEST PUBLIC:
   ¿Son idénticas? True

📊 TEST PRIVATE:
   ¿Son idénticas? True

DECISIÓN

🚨 COLUMNAS DUPLICADAS en TODOS los datasets
   → Eliminando 'incumplimiento'...
   ✅ Columna 'incumplimiento' eliminada de todos los datasets
   ✅ Mantenida columna 'tiene_riesgo'


### Preprocesar columna "contacto"

In [29]:
"""
=================================================================
PREPROCESAMIENTO - COLUMNA 'contacto'
=================================================================
"""

def limpiar_contacto(df):
    """
    Limpia y estandariza la columna 'contacto'

    Unifica:
    - CELL, CEL, celular, Celular → celular
    - teléfono, TEL, Teléfono → telefono
    - Desconocido, desconocido → desconocido
    """
    df = df.copy()

    # Normalizar a minúsculas
    df['contacto'] = df['contacto'].str.lower().str.strip()

    # Mapeo de estandarización
    mapeo = {
        # Variaciones de "celular"
        'cell': 'celular',
        'cel': 'celular',

        # Variaciones de "teléfono"
        'teléfono': 'telefono',  # Quitar tilde
        'tel': 'telefono',
    }

    df['contacto'] = df['contacto'].replace(mapeo)

    return df

# ========== APLICAR ==========
train_clean = limpiar_contacto(train_clean)
test_public_clean = limpiar_contacto(test_public_clean)
test_private_clean = limpiar_contacto(test_private_clean)

# ========== VERIFICAR ==========
print("="*60)
print("LIMPIEZA COLUMNA 'contacto' - COMPLETADA")
print("="*60)

print(f"\n📊 Categorías: 9 → {train_clean['contacto'].nunique()}")
print("\n📋 Distribución final:")
print(train_clean['contacto'].value_counts())

# Verificar valores esperados
valores_esperados = {'celular', 'telefono', 'desconocido'}
valores_reales = set(train_clean['contacto'].unique())

if valores_reales == valores_esperados:
    print("\n✅ Limpieza exitosa - 3 categorías correctas")
else:
    print(f"\n⚠️  Valores inesperados: {valores_reales}")

# Distribución porcentual
print("\n📊 Distribución porcentual:")
print(train_clean['contacto'].value_counts(normalize=True).sort_values(ascending=False) * 100)

LIMPIEZA COLUMNA 'contacto' - COMPLETADA

📊 Categorías: 9 → 3

📋 Distribución final:
contacto
celular        23464
desconocido    10385
telefono        2319
Name: count, dtype: int64

✅ Limpieza exitosa - 3 categorías correctas

📊 Distribución porcentual:
contacto
celular        64.875028
desconocido    28.713227
telefono        6.411745
Name: proportion, dtype: float64


### Preprocesar columna "p_resultado"

In [30]:
"""
=================================================================
PREPROCESAMIENTO - COLUMNA 'p_resultado'
=================================================================
"""

def limpiar_p_resultado(df):
    """
    Limpia y estandariza la columna 'p_resultado'

    Unifica:
    - D, Desconocido, desconocido → desconocido
    - F, fracaso, Fracaso → fracaso
    - E, Éxito, éxito → exito
    - O, Otro, otro → otro
    """
    df = df.copy()

    # Normalizar a minúsculas
    df['p_resultado'] = df['p_resultado'].str.lower().str.strip()

    # Quitar tildes para estandarizar
    df['p_resultado'] = df['p_resultado'].str.normalize('NFKD')\
                                          .str.encode('ascii', errors='ignore')\
                                          .str.decode('utf-8')

    # Mapeo de abreviaciones
    mapeo = {
        'd': 'desconocido',
        'f': 'fracaso',
        'e': 'exito',
        'o': 'otro',
    }

    df['p_resultado'] = df['p_resultado'].replace(mapeo)

    return df

# ========== APLICAR ==========
train_clean = limpiar_p_resultado(train_clean)
test_public_clean = limpiar_p_resultado(test_public_clean)
test_private_clean = limpiar_p_resultado(test_private_clean)

# ========== VERIFICAR ==========
print("="*60)
print("LIMPIEZA COLUMNA 'p_resultado' - COMPLETADA")
print("="*60)

print(f"\n📊 Categorías: 12 → {train_clean['p_resultado'].nunique()}")
print("\n📋 Distribución final:")
print(train_clean['p_resultado'].value_counts())

# Verificar valores esperados
valores_esperados = {'desconocido', 'fracaso', 'exito', 'otro'}
valores_reales = set(train_clean['p_resultado'].unique())

if valores_reales == valores_esperados:
    print("\n✅ Limpieza exitosa - 4 categorías correctas")
else:
    print(f"\n⚠️  Valores inesperados: {valores_reales}")

# Distribución porcentual
print("\n📊 Distribución porcentual:")
dist = train_clean['p_resultado'].value_counts(normalize=True) * 100
print(dist.sort_values(ascending=False))

LIMPIEZA COLUMNA 'p_resultado' - COMPLETADA

📊 Categorías: 12 → 4

📋 Distribución final:
p_resultado
desconocido    29560
fracaso         3901
otro            1497
exito           1210
Name: count, dtype: int64

✅ Limpieza exitosa - 4 categorías correctas

📊 Distribución porcentual:
p_resultado
desconocido    81.729706
fracaso        10.785777
otro            4.139018
exito           3.345499
Name: proportion, dtype: float64


# AAA